In [1]:
import cv2
from ultralytics import YOLO
import numpy as np
from collections import deque

In [2]:
# Cargar el modelo YOLO 
model = YOLO('yolov8n.pt')

100%|██████████| 6.23M/6.23M [00:16<00:00, 388kB/s] 


In [3]:
# Definir un diccionario de colores para las clases de objetos
class_colors = {
    'person': (255, 0, 0),  # Rojo
    'car': (0, 255, 0),     # Verde
    'dog': (0, 0, 255),     # Azul
    # Agregar más clases y colores según sea necesario
}

In [ ]:
#Versión uno
def draw_boxes(frame, results):
    for result in results:
        boxes = result.boxes
        for box in boxes:
            # Obtener las coordenadas de la caja
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0]
            cls = box.cls[0]

            # Verificar si la clase está en result.names
            if int(cls) in result.names:
                label = result.names[int(cls)]
            else:
                label = f'cls {int(cls)}'

            # Dibujar la caja y la etiqueta en el frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    return frame

In [ ]:
#Versión 2
def draw_boxes(frame, results, confidence_threshold=0.5):
    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = box.conf[0]
            
            # Mostrar solo las detecciones con una confianza superior al umbral
            if conf > confidence_threshold:
                # Obtener las coordenadas de la caja
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = box.cls[0]

                # Verificar si la clase está en result.names
                if int(cls) in result.names:
                    label = result.names[int(cls)]
                else:
                    label = f'cls {int(cls)}'

                # Dibujar la caja y la etiqueta en el frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    return frame


In [ ]:
#Versión 3
# Definir un diccionario de colores para las clases de objetos
class_colors = {
    'person': (255, 0, 0),  # Rojo
    'cat': (0, 255, 0),     # Verde
    'dog': (0, 0, 255),     # Azul
    # Agregar más clases y colores según sea necesario
}
def draw_boxes(frame, results, confidence_threshold=0.5):
    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = box.conf[0]
            
            # Mostrar solo las detecciones con una confianza superior al umbral
            if conf > confidence_threshold:
                # Obtener las coordenadas de la caja
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = box.cls[0]

                # Verificar si la clase está en result.names
                if int(cls) in result.names:
                    label = result.names[int(cls)]
                    # Obtener el color para la clase actual
                    color = class_colors.get(label, (255, 255, 255))  # Blanco por defecto si la clase no está en el diccionario
                else:
                    label = f'cls {int(cls)}'
                    color = (255, 255, 255)  # Blanco por defecto para clases desconocidas

                # Dibujar la caja y la etiqueta en el frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    return frame



In [ ]:
#Versión 4 distancia euclidiana
# Definir un diccionario de colores para las clases de objetos
class_colors = {
    'person': (255, 0, 0),  # Rojo
    'cat': (0, 255, 0),     # Verde
    'dog': (0, 0, 255),     # Azul
    # Agregar más clases y colores según sea necesario
}
def draw_boxes(frame, results, confidence_threshold=0.5):
    frame_height, frame_width = frame.shape[:2]
    center_x, center_y = frame_width // 2, frame_height // 2
    
    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = box.conf[0]
            
            # Mostrar solo las detecciones con una confianza superior al umbral
            if conf > confidence_threshold:
                # Obtener las coordenadas de la caja
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = box.cls[0]

                # Verificar si la clase está en result.names
                if int(cls) in result.names:
                    label = result.names[int(cls)]
                    # Obtener el color para la clase actual
                    color = class_colors.get(label, (255, 255, 255))  # Blanco por defecto si la clase no está en el diccionario
                else:
                    label = f'cls {int(cls)}'
                    color = (255, 255, 255)  # Blanco por defecto para clases desconocidas

                # Calcular el centro del recuadro del objeto
                object_center_x = (x1 + x2) // 2
                object_center_y = (y1 + y2) // 2

                # Calcular la distancia euclidiana desde el centro del frame al centro del recuadro del objeto
                distance = int(np.sqrt((object_center_x - center_x)**2 + (object_center_y - center_y)**2))

                # Dibujar la caja y la etiqueta en el frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{label} {conf:.2f} Dist: {distance}px", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                
                # Dibujar una línea desde el centro del frame al centro del objeto
                cv2.line(frame, (center_x, center_y), (object_center_x, object_center_y), color, 2)

    return frame


In [ ]:
#version 5
def draw_boxes(frame, results, confidence_threshold=0.5):
    frame_height, frame_width = frame.shape[:2]
    center_x = frame_width // 2  # Centro del eje x
    origin_y = 0  # Parte superior del frame

    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = box.conf[0]
            
            # Mostrar solo las detecciones con una confianza superior al umbral
            if conf > confidence_threshold:
                # Obtener las coordenadas de la caja
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = box.cls[0]

                # Verificar si la clase está en result.names
                if int(cls) in result.names:
                    label = result.names[int(cls)]
                    # Obtener el color para la clase actual
                    color = class_colors.get(label, (255, 255, 255))  # Blanco por defecto si la clase no está en el diccionario
                else:
                    label = f'cls {int(cls)}'
                    color = (255, 255, 255)  # Blanco por defecto para clases desconocidas

                # Calcular el centro del recuadro del objeto
                object_center_x = (x1 + x2) // 2
                object_center_y = (y1 + y2) // 2

                # Calcular la distancia euclidiana desde el punto (center_x, origin_y) al centro del recuadro del objeto
                distance = int(np.sqrt((object_center_x - center_x)**2 + (object_center_y - origin_y)**2))

                # Dibujar la caja y la etiqueta en el frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{label} {conf:.2f} Dist: {distance}px", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                
                # Dibujar una línea desde el punto (center_x, origin_y) al centro del objeto
                cv2.line(frame, (center_x, origin_y), (object_center_x, object_center_y), color, 2)

    return frame


In [ ]:
#Versión 6
def draw_boxes(frame, results, confidence_threshold=0.5):
    frame_height, frame_width = frame.shape[:2]
    center_x = frame_width // 2  # Centro del eje x
    bottom_y = frame_height      # Parte inferior del frame

    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = box.conf[0]
            
            # Mostrar solo las detecciones con una confianza superior al umbral
            if conf > confidence_threshold:
                # Obtener las coordenadas de la caja
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = box.cls[0]

                # Verificar si la clase está en result.names
                if int(cls) in result.names:
                    label = result.names[int(cls)]
                    # Obtener el color para la clase actual
                    color = class_colors.get(label, (255, 255, 255))  # Blanco por defecto si la clase no está en el diccionario
                else:
                    label = f'cls {int(cls)}'
                    color = (255, 255, 255)  # Blanco por defecto para clases desconocidas

                # Calcular el centro del recuadro del objeto
                object_center_x = (x1 + x2) // 2
                object_center_y = (y1 + y2) // 2

                # Calcular la distancia euclidiana desde el punto (center_x, bottom_y) al centro del recuadro del objeto
                distance = int(np.sqrt((object_center_x - center_x)**2 + (object_center_y - bottom_y)**2))

                # Dibujar la caja y la etiqueta en el frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{label} {conf:.2f} Dist: {distance}px", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                
                # Dibujar una línea desde el punto (center_x, bottom_y) al centro del objeto
                cv2.line(frame, (center_x, bottom_y), (object_center_x, object_center_y), color, 2)

    return frame


In [ ]:
#versión 7
def draw_boxes(frame, results, confidence_threshold=0.5):
    frame_height, frame_width = frame.shape[:2]
    center_x = frame_width // 2  # Centro del eje x
    bottom_y = frame_height      # Parte inferior del frame

    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = box.conf[0]
            
            # Mostrar solo las detecciones con una confianza superior al umbral
            if conf > confidence_threshold:
                # Obtener las coordenadas de la caja
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = box.cls[0]

                # Verificar si la clase está en result.names
                if int(cls) in result.names:
                    label = result.names[int(cls)]
                    # Obtener el color para la clase actual
                    color = class_colors.get(label, (255, 255, 255))  # Blanco por defecto si la clase no está en el diccionario
                else:
                    label = f'cls {int(cls)}'
                    color = (255, 255, 255)  # Blanco por defecto para clases desconocidas

                # Calcular el centro del recuadro del objeto
                object_center_x = (x1 + x2) // 2
                object_center_y = (y1 + y2) // 2

                # Calcular la distancia euclidiana desde el punto (center_x, bottom_y) al centro del recuadro del objeto
                distance = int(np.sqrt((object_center_x - center_x)**2 + (object_center_y - bottom_y)**2))

                # Dibujar la caja y la etiqueta en el frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{label} {conf:.2f} Dist: {distance}px", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                
                # Dibujar una línea desde el punto (center_x, bottom_y) al centro del objeto
                cv2.line(frame, (center_x, bottom_y), (object_center_x, object_center_y), color, 2)

    return frame


In [ ]:
#veersion 8
def draw_boxes(frame, results, confidence_threshold=0.5):
    frame_height, frame_width = frame.shape[:2]
    center_x = frame_width // 2  # Centro del eje x
    bottom_y = frame_height      # Parte inferior del frame

    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = box.conf[0]
            
            # Mostrar solo las detecciones con una confianza superior al umbral
            if conf > confidence_threshold:
                # Obtener las coordenadas de la caja
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = box.cls[0]

                # Verificar si la clase está en result.names
                if int(cls) in result.names:
                    label = result.names[int(cls)]
                    # Obtener el color para la clase actual
                    color = class_colors.get(label, (255, 255, 255))  # Blanco por defecto si la clase no está en el diccionario
                else:
                    label = f'cls {int(cls)}'
                    color = (255, 255, 255)  # Blanco por defecto para clases desconocidas

                # Calcular el centro del recuadro del objeto
                object_center_x = (x1 + x2) // 2
                object_center_y = (y1 + y2) // 2

                # Calcular la distancia inversa
                distance = int(np.sqrt((object_center_x - center_x)**2 + (bottom_y - object_center_y)**2))

                # Dibujar la caja y la etiqueta en el frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{label} {conf:.2f} Dist: {distance}px", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                
                # Dibujar una línea desde el punto (center_x, bottom_y) al centro del objeto
                cv2.line(frame, (center_x, bottom_y), (object_center_x, object_center_y), color, 2)

    return frame


In [ ]:
#Version 9
def draw_boxes(frame, results, confidence_threshold=0.4):
    frame_height, frame_width = frame.shape[:2]
    center_x = frame_width // 2  # Centro del eje x
    bottom_y = frame_height      # Parte inferior del frame

    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = box.conf[0]
            
            # Mostrar solo las detecciones con una confianza superior al umbral
            if conf > confidence_threshold:
                # Obtener las coordenadas de la caja
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = box.cls[0]

                # Verificar si la clase está en result.names
                if int(cls) in result.names:
                    label = result.names[int(cls)]
                    # Obtener el color para la clase actual
                    color = class_colors.get(label, (255, 255, 255))  # Blanco por defecto si la clase no está en el diccionario
                else:
                    label = f'cls {int(cls)}'
                    color = (255, 255, 255)  # Blanco por defecto para clases desconocidas

                # Calcular el centro del recuadro del objeto
                object_center_x = (x1 + x2) // 2
                object_center_y = (y1 + y2) // 2

                # Calcular la distancia inversa
                distance = int(np.sqrt((object_center_x - center_x)**2 + (bottom_y - object_center_y)**2))

                # Dibujar la caja y la etiqueta en el frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                label_text = f"{label} {conf:.2f} Dist: {distance}px"
                coord_text = f"({object_center_x}, {object_center_y})"
                cv2.putText(frame, label_text, (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                cv2.putText(frame, coord_text, (x1, y1 - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                
                # Dibujar una línea desde el punto (center_x, bottom_y) al centro del objeto
                cv2.line(frame, (center_x, bottom_y), (object_center_x, object_center_y), color, 2)

    return frame


In [ ]:
#Version 10
# Diccionario para almacenar las trayectorias de los objetos
trajectories = {}

def draw_boxes(frame, results, confidence_threshold=0.5):
    frame_height, frame_width = frame.shape[:2]
    center_x = frame_width // 2  # Centro del eje x
    bottom_y = frame_height      # Parte inferior del frame

    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = box.conf[0]
            
            # Mostrar solo las detecciones con una confianza superior al umbral
            if conf > confidence_threshold:
                # Obtener las coordenadas de la caja
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = box.cls[0]

                # Verificar si la clase está en result.names
                if int(cls) in result.names:
                    label = result.names[int(cls)]
                    # Obtener el color para la clase actual
                    color = class_colors.get(label, (255, 255, 255))  # Blanco por defecto si la clase no está en el diccionario
                else:
                    label = f'cls {int(cls)}'
                    color = (255, 255, 255)  # Blanco por defecto para clases desconocidas

                # Calcular el centro del recuadro del objeto
                object_center_x = (x1 + x2) // 2
                object_center_y = (y1 + y2) // 2

                # Calcular la distancia inversa
                distance = int(np.sqrt((object_center_x - center_x)**2 + (bottom_y - object_center_y)**2))

                # Dibujar la caja y la etiqueta en el frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                label_text = f"{label} {conf:.2f} Dist: {distance}px"
                coord_text = f"({object_center_x}, {object_center_y})"
                cv2.putText(frame, label_text, (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                cv2.putText(frame, coord_text, (x1, y1 - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                
                # Dibujar una línea desde el punto (center_x, bottom_y) al centro del objeto
                cv2.line(frame, (center_x, bottom_y), (object_center_x, object_center_y), color, 2)

                # Actualizar las trayectorias
                if label not in trajectories:
                    trajectories[label] = deque(maxlen=64)  # Puedes ajustar la longitud máxima de la trayectoria
                trajectories[label].append((object_center_x, object_center_y))

                # Dibujar la trayectoria
                for i in range(1, len(trajectories[label])):
                    if trajectories[label][i - 1] is None or trajectories[label][i] is None:
                        continue
                    thickness = int(np.sqrt(64 / float(i + 1)) * 2.5)
                    cv2.line(frame, trajectories[label][i - 1], trajectories[label][i], color, thickness)

    return frame


In [ ]:
#Versión 11
def draw_boxes_and_triangle(frame, results, confidence_threshold=0.5):
    frame_height, frame_width = frame.shape[:2]
    center_x = frame_width // 2  # Centro del eje x
    bottom_y = frame_height      # Parte inferior del frame
    half_y = frame_height // 2   # 50% del eje y

    # Coordenadas del triángulo
    triangle_vertices = np.array([[center_x, bottom_y], [center_x - center_x, half_y], [center_x + center_x, half_y]], np.int32)
    triangle_vertices = triangle_vertices.reshape((-1, 1, 2))

    # Dibujar el triángulo
    cv2.polylines(frame, [triangle_vertices], isClosed=True, color=(0, 255, 255), thickness=2)

    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = box.conf[0]
            
            # Mostrar solo las detecciones con una confianza superior al umbral
            if conf > confidence_threshold:
                # Obtener las coordenadas de la caja
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = box.cls[0]

                # Verificar si la clase está en result.names
                if int(cls) in result.names:
                    label = result.names[int(cls)]
                    # Obtener el color para la clase actual
                    color = class_colors.get(label, (255, 255, 255))  # Blanco por defecto si la clase no está en el diccionario
                else:
                    label = f'cls {int(cls)}'
                    color = (255, 255, 255)  # Blanco por defecto para clases desconocidas

                # Calcular el centro del recuadro del objeto
                object_center_x = (x1 + x2) // 2
                object_center_y = (y1 + y2) // 2

                # Calcular la distancia inversa
                distance = int(np.sqrt((object_center_x - center_x)**2 + (bottom_y - object_center_y)**2))

                # Dibujar la caja y la etiqueta en el frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                label_text = f"{label} {conf:.2f} Dist: {distance}px"
                coord_text = f"({object_center_x}, {object_center_y})"
                cv2.putText(frame, label_text, (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                cv2.putText(frame, coord_text, (x1, y1 - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                
                # Dibujar una línea desde el punto (center_x, bottom_y) al centro del objeto
                cv2.line(frame, (center_x, bottom_y), (object_center_x, object_center_y), color, 2)

    return frame


In [ ]:
#Versión 12
def draw_boxes_and_triangle(frame, results, confidence_threshold=0.5):
    frame_height, frame_width = frame.shape[:2]
    center_x = frame_width // 2  # Centro del eje x
    bottom_y = frame_height      # Parte inferior del frame
    quarter_x = frame_width // 4  # 25% del eje x
    three_quarters_x = 3 * frame_width // 4  # 75% del eje x
    half_y = frame_height // 2   # 50% del eje y

    # Coordenadas del triángulo con base reducida a la mitad
    triangle_vertices = np.array([[center_x, bottom_y], [center_x - quarter_x, half_y], [center_x + quarter_x, half_y]], np.int32)
    triangle_vertices = triangle_vertices.reshape((-1, 1, 2))

    # Dibujar el triángulo
    cv2.polylines(frame, [triangle_vertices], isClosed=True, color=(0, 255, 255), thickness=2)

    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = box.conf[0]
            
            # Mostrar solo las detecciones con una confianza superior al umbral
            if conf > confidence_threshold:
                # Obtener las coordenadas de la caja
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = box.cls[0]

                # Verificar si la clase está en result.names
                if int(cls) in result.names:
                    label = result.names[int(cls)]
                    # Obtener el color para la clase actual
                    color = class_colors.get(label, (255, 255, 255))  # Blanco por defecto si la clase no está en el diccionario
                else:
                    label = f'cls {int(cls)}'
                    color = (255, 255, 255)  # Blanco por defecto para clases desconocidas

                # Calcular el centro del recuadro del objeto
                object_center_x = (x1 + x2) // 2
                object_center_y = (y1 + y2) // 2

                # Calcular la distancia inversa
                distance = int(np.sqrt((object_center_x - center_x)**2 + (bottom_y - object_center_y)**2))

                # Dibujar la caja y la etiqueta en el frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                label_text = f"{label} {conf:.2f} Dist: {distance}px"
                coord_text = f"({object_center_x}, {object_center_y})"
                cv2.putText(frame, label_text, (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                cv2.putText(frame, coord_text, (x1, y1 - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                
                # Dibujar una línea desde el punto (center_x, bottom_y) al centro del objeto
                cv2.line(frame, (center_x, bottom_y), (object_center_x, object_center_y), color, 2)

    return frame


In [10]:
#Versión 13
def point_in_triangle(pt, v1, v2, v3):
    """Verifica si el punto pt está dentro del triángulo definido por v1, v2, v3."""
    d1 = sign(pt, v1, v2) < 0.0
    d2 = sign(pt, v2, v3) < 0.0
    d3 = sign(pt, v3, v1) < 0.0
    return d1 == d2 == d3

def sign(pt1, pt2, pt3):
    return (pt1[0] - pt3[0]) * (pt2[1] - pt3[1]) - (pt2[0] - pt3[0]) * (pt1[1] - pt3[1])


In [11]:
#Version 13A
def draw_boxes_and_triangle(frame, results, triangle_vertices, confidence_threshold=0.5):
    frame_height, frame_width = frame.shape[:2]

    # Asegurarse de que las coordenadas del triángulo sean tuplas
    v1, v2, v3 = tuple(triangle_vertices[0][0]), tuple(triangle_vertices[1][0]), tuple(triangle_vertices[2][0])

    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = box.conf[0]
            
            # Mostrar solo las detecciones con una confianza superior al umbral
            if conf > confidence_threshold:
                # Obtener las coordenadas de la caja
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = box.cls[0]

                # Verificar si la clase está en result.names
                if int(cls) in result.names:
                    label = result.names[int(cls)]
                    # Obtener el color para la clase actual
                    color = class_colors.get(label, (255, 255, 255))  # Blanco por defecto si la clase no está en el diccionario
                else:
                    label = f'cls {int(cls)}'
                    color = (255, 255, 255)  # Blanco por defecto para clases desconocidas

                # Calcular el centro del recuadro del objeto
                object_center_x = (x1 + x2) // 2
                object_center_y = (y1 + y2) // 2

                # Verificar si el centro del objeto está dentro del triángulo
                if point_in_triangle((object_center_x, object_center_y), v1, v2, v3):
                    # Cambiar el color a rojo y mostrar "Warning" en la etiqueta
                    color = (0, 0, 255)  # Rojo
                    label = "Warning"

                # Calcular la distancia inversa
                distance = int(np.sqrt((object_center_x - v1[0])**2 + (object_center_y - v1[1])**2))

                # Dibujar la caja y la etiqueta en el frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                label_text = f"{label} {conf:.2f} Dist: {distance}px"
                coord_text = f"({object_center_x}, {object_center_y})"
                cv2.putText(frame, label_text, (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                cv2.putText(frame, coord_text, (x1, y1 - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                
                # Dibujar una línea desde el punto (center_x, bottom_y) al centro del objeto
                cv2.line(frame, (v1[0], v1[1]), (object_center_x, object_center_y), color, 2)

    # Dibujar el triángulo
    cv2.polylines(frame, [triangle_vertices], isClosed=True, color=(0, 255, 255), thickness=2)

    return frame


In [14]:
# Versión 13B
# Capturar video desde una cámara o un archivo de video
cap = cv2.VideoCapture('moto.mp4')  # Reemplaza '0' por el archivo de video si es necesario

# Definir las dimensiones personalizadas
custom_width = 1500
custom_height = 700

# Coordenadas del triángulo con base reducida a la mitad
center_x = custom_width // 2
bottom_y = custom_height
half_y = custom_height // 2
quarter_x = custom_width // 4
triangle_vertices = np.array([[center_x, bottom_y], [center_x - quarter_x, half_y], [center_x + quarter_x, half_y]], np.int32)
triangle_vertices = triangle_vertices.reshape((-1, 1, 2))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Redimensionar el frame al tamaño personalizado
    frame = cv2.resize(frame, (custom_width, custom_height))

    # Realizar la detección con YOLO
    results = model(frame)

    # Dibujar las cajas de detección en el frame
    frame = draw_boxes_and_triangle(frame, results, triangle_vertices, confidence_threshold=0.4)  # Establecer el umbral de confianza deseado

    # Mostrar el frame con las detecciones
    cv2.imshow('YOLO Object Tracking', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 320x640 (no detections), 124.0ms
Speed: 0.0ms preprocess, 124.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 (no detections), 116.3ms
Speed: 5.3ms preprocess, 116.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 (no detections), 105.1ms
Speed: 0.0ms preprocess, 105.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 (no detections), 102.3ms
Speed: 0.0ms preprocess, 102.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 (no detections), 105.7ms
Speed: 1.0ms preprocess, 105.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 (no detections), 98.2ms
Speed: 1.0ms preprocess, 98.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 (no detections), 101.0ms
Speed: 2.3ms preprocess, 101.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 (no detections), 118.1ms
Speed: 0.0ms preproc

In [9]:
# Capturar video desde una cámara o un archivo de video
cap = cv2.VideoCapture('moto.mp4')  # Reemplaza '0' por el archivo de video si es necesario

# Definir las dimensiones personalizadas
custom_width = 1000
custom_height = 405

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Redimensionar el frame al tamaño personalizado
    frame = cv2.resize(frame, (custom_width, custom_height))

    # Realizar la detección con YOLO
    results = model(frame)

    # Dibujar las cajas de detección en el frame
    frame = draw_boxes_and_triangle(frame, results, confidence_threshold=0.5)  # Establecer el umbral de confianza deseado

    # Mostrar el frame con las detecciones
    cv2.imshow('YOLO Object Tracking', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 288x640 (no detections), 180.0ms
Speed: 4.6ms preprocess, 180.0ms inference, 11.5ms postprocess per image at shape (1, 3, 640, 640)


TypeError: draw_boxes_and_triangle() missing 1 required positional argument: 'triangle_vertices'